# 2. laboratorijska vježba

In [ ]:
# učitvanje potrebnih biblioteka

import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as ss

In [ ]:
#@title pomoćna funkcija
# izvršite ovu ćeliju ali se ne opterećujte detaljima implementacije

def plot_frequency_response(f, Hm, fc=None, ylim_min=None):
    """Grafički prikaz prijenosne funkcije filtra.
    
    Args
        f (numpy.ndarray) : frekvencije
        Hm (numpy.ndarray) : apsolutne vrijednosti prijenosne funkcije
        fc (number) : cutoff frekvencija
        ylim_min (number): minimalna vrijednost na y-osi za dB skalu

    Returns
        (matplotlib.figure.Figure, matplotlib.axes._subplots.AxesSubplot)
    """
    Hc = 1 / np.sqrt(2)
    if fc is None:
        fc_idx = np.where(np.isclose(Hm, Hc, rtol=1e-03))[0][0]
        fc = f[fc_idx]
    H_db = 20 * np.log10(Hm)
    
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12, 7.5))

    ax[0, 0].plot(f, Hm, label='$H(f)$')
    ax[0, 0].plot(fc, Hc, 'o', label='$H(f_c)$')
    ax[0, 0].vlines(fc, Hm.min(), Hc, linestyle='--')
    ax[0, 0].annotate(f'$f_c = {fc:.3f}$ Hz\n$H(f_c)={Hc:.3f}$', (fc * 1.4, Hc))
    ax[0, 0].set_xscale('log')
    ax[0, 0].set_ylabel('$|V_{out}$ / $V_{in}$|')
    ax[0, 0].set_title('log scale')
    ax[0, 0].legend(loc='lower left')
    ax[0, 0].grid()
    
    ax[0, 1].plot(f, Hm, label='$H(f)$')
    ax[0, 1].plot(fc, Hc, 'o', label='$H(f_c)$')
    ax[0, 1].annotate(f'$f_c = {fc:.3f}$ Hz\n$H(f_c)={Hc:.3f}$', (fc * 1.4, Hc))
    ax[0, 1].set_title('linear scale')
    ax[0, 1].legend()
    ax[0, 1].grid()

    ax[1, 0].plot(f, H_db, label='$H_{dB}(f)$')
    ax[1, 0].plot(fc, H_db.max() - 3, 'o', label='$H_{dB}(f_c)$')
    ax[1, 0].vlines(fc, H_db.min(), H_db.max() - 3, linestyle='--')
    ax[1, 0].annotate(f'$f_c = {fc:.3f}$ Hz\n$H(f_c)={H_db.max() - 3:.3f} dB$',
                      (fc * 1.4, H_db.max() - 3))
    ax[1, 0].set_xscale('log')
    ax[1, 0].set_xlabel('$f$ [Hz]')
    ax[1, 0].set_ylabel('$20 \\cdot \\log$ |$V_{out}$ / $V_{in}$|')
    if ylim_min:
        ax[1, 0].set_ylim((ylim_min, 10))
    ax[1, 0].legend(loc='lower left')
    ax[1, 0].grid()

    ax[1, 1].plot(f, H_db, label='$H_{dB}(f)$')
    ax[1, 1].plot(fc, H_db.max() - 3, 'o', label='$H_{dB}(f_c)$')
    ax[1, 1].annotate(f'$f_c = {fc:.3f}$ Hz\n$H(f_c)={H_db.max() - 3:.3f} dB$',
                      (fc * 1.4, H_db.max() - 3))
    ax[1, 1].set_xlabel('$f$ [Hz]')
    if ylim_min:
        ax[1, 1].set_ylim((ylim_min, 10))
    ax[1, 1].legend()
    ax[1, 1].grid()

    fig.tight_layout
    return fig, ax

## Električni filtri

Općenito, filtar u domeni elektromagnetske kompatibilnosti, kao i kroz ovaj kolegij, promatramo kao model električnog kruga definiranog koristeći koncentrirane parametre. Signal s ulaza se oblikuje na način da njegove komponente pri određenim frekvencijama filtar propušta bez izobličenja (u teoriji), dok na drugim frekvencijama komponente uopće ne propušta (u teoriji, u stvarnosti dolazi do jače ili slabije atenuacije). Selektivno propuštanje signala se radi ponajprije kako bi se potisnule ili u potpunosti otklonile neželjene komponente signala.

S obzirom na frekvencijski pojas propusta, filtre dijelimo u osnovne kategorije:
1. nisko-propusni filtri - propuštaju se sve frekvencije signala ispod tzv. *cutoff* frekvencije;
2. visoko-propusni filtri - propuštaju se sve frekvencije signala iznad *cutoff* frekvencije;
3. pojasno-propusni filtri - propušta se definirani frekvencijski opseg;
4. pojasno-nepropusni filtri - ne propušta se određeni frekvencijski opseg.

S obzirom na tip koncentriranih parametara, odnosno komponenti koje koristimo kako bismo računalno modelirali rad filtra, filtre dijelimo na dodatne kategorije:
1. aktivni filtri - realizirani uz pomoć aktivnih komponenti (tranzistori i operacijska pojačala);
2. pasivni filtri - realizirani uz pomoć pasivnih komponenti (otpornici, kondenzatori i zavojnice).

### Pasivni nisko-propusni filtar

Realizacija nisko-propusnog filtra u ovim laboratorijskim vježbama se ostvaruje korištenjem otpornika i kondenzatora povezanih u seriju, pri čemu se izlaz promatra kao napon na kondenzatoru, $V_{out}$. Uz pretpostavku da je signal na ulazu, $V_{in}$, sinusoidalni naponski izvor, analizu možemo prebaciti u frekvencijsku domenu koristeći impedancijski model. Na ovaj način zaobilazimo potrebu korištenja diferencijalnog i čitav problem svodimo na jednostavni algebarski proračun.

<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e0/1st_Order_Lowpass_Filter_RC.svg/1920px-1st_Order_Lowpass_Filter_RC.svg.png" alt="simple-rc-lowpass" width="300"/>
</center>
    
Napon na kondenzatoru, $V_{out}$, definiramo kroz podjelu ulaznog napona:

$$
\begin{align}
    V_{out} &= \frac{Z_c}{Z_c + Z_r} \cdot V_{in} \\
    \frac{V_{out}}{V_{in}} &= \frac{Z_c}{Z_c + Z_r} = \frac{1/(j\omega C)}{1/(j\omega C) + R} = \frac{1}{1 + j\omega C R}
\end{align}
$$

Dobiveni odnos izlaznog i ulaznog napona se naziva **funkcija prijenosnog odziva**, $H$, u stacionarnom stanju:

$$
H(\omega) = \frac{V_{out}}{V_{in}} = \frac{1}{1 + j\omega C R}
$$

Kako je $H$ funkcija frekvencije, imamo dva rubna slučaja:
* za iznimno niske frekvencije kada je $\omega \sim 0$ slijedi da je $H(\omega) = 1$;
* za iznimno visoke frekvencije kada $\omega \rightarrow \infty$ slijedi da je $H(\omega) \rightarrow 0$.

Potrebno je dodatno definirati već spomenutu *cutoff* frekvenciju, $f_c$, za koju amplituda funkcije frekvencijskog odziva, $H$, pada za $\sqrt 2$ puta, odnosno za $3$ dB:

$$
\begin{align}
    \frac{H(\omega)}{\sqrt 2} &= \frac{1}{1 + j\omega_c C R} \\
    \omega_c &= \frac{1}{CR} \\
    f_c &= \frac{1}{2\pi CR}
\end{align}
$$

Link za interaktivni rad s pasivnim nisko-propusnim filtrom: http://sim.okawa-denshi.jp/en/CRtool.php

#### Zadatak 1

Prvi zadatak je implementirati funkciju `cutoff_frequency` koja na ulazu prima iznose otpora, `R`, i kapaciteta, `C`, a na izlazu daje *cutoff* frekvenciju nisko-propusnog filtra.

In [ ]:
def cutoff_frequency(R, C):
    """Cutoff frekvencija nisko-propusnog filtra.
    
    Args:
        R (number) : vrijednost otpora otpornika
        C (number) : kapacitet kondenzatora
    
    Returns:
        number
    """
    #######################################################
    ## TO-DO: implementiraj proračun cutoff frekvencije ##
    # Nakon toga zakomentiraj sljedeću liniju.
    raise NotImplementedError('Implementiraj proračun cutoff frekvencije.')
    #######################################################

    # definiraj cutoff frekvenciju
    fc = ...
    return fc

Kolika je *cutoff* frekvencija za otpor od $4.5 k \Omega$ i kapacitet kondenzatora od $7 \mu F$?

In [ ]:
R = ...  # otpor u Ohm
C = ...  # kapacitet u F

fc = ...  # cutoff frekvencija

print(f'R = {R / 1000:.2f} kΩ')
print(f'C = {C * 10**6:.2f} µF')
print(f'cutoff frekvencija iznosi {fc:.2f} Hz, '
      'očekivana vrijednost je 5.05 Hz')

#### Zadatak 2

Drugi zadatak je implementirati funkciju `rc_lowpass` koja na ulazu prima iznose otpora, `R`, kapaciteta, `C`, i frekvenciju, `f`, a na izlazu daje prijenosni odziv nisko-propusnog filtra.

In [ ]:
def rc_lowpass(R, C, f):
    """Funkcija prijenosnog odziva RC nisko-propusnog filtra.
    
    Args:
        R (number) : vrijednost otpora otpornika
        C (number) : kapacitet kondenzatora
        f (number or numpy.ndarray) : frekvencija/e
    
    Returns:
        float or numpy.ndarray
    """
    ######################################################
    ## TO-DO: implementiraj funkciju prijenosnog odziva ##
    # Nakon toga zakomentiraj sljedeću liniju.
    raise NotImplementedError('Implementiraj funckiju prijenosnog odziva.')
    ######################################################

    # definiraj funkciju prijenosnog pazeći da `f` može biti ili broj (int,
    # float) ili 1-D niz (`numpy.ndarray`)
    H = ...
    return H

Kolika je vrijednost prijenosne funkcije pri *cutoff* frekvencija za otpor od $4.5 k \Omega$ i kapacitet kondenzatora od $7 \mu F$?

In [ ]:
R = ...  # otpor u Ohm
C = ...  # kapacitet u F

Hc = ...  # prijenosna funkcija pri cutoff frekvenciji

print(f'R = {R / 1000:.2f} kΩ')
print(f'C = {C * 10**6:.2f} µF')
print(f'pojačanje pri cutoff frekvenciji iznosi {abs(Hc):.4f}, '
      'očekivana vrijednost je 1/√2\n\n'
      'provjerite ispravnost dobivenog rezutltata')

In [ ]:
# ćelija za provjeru rezultata



Pretvorite vrijednost prijenosne funkcije pri *cutoff* frekvenciju u decibele i uvjerite se u tvrdnju da amplituda funkcije frekvencijskog odziva, $H$, pada za $3$ dB pri *cutoff* frekvenciji.

In [ ]:
Hc_dB = ...  # pretvorba prijenosne funkcije pri cutoff frekvenciji u dB skalu
print(Hc_dB)

Za raspon od $10000$ vrijednosti frekvencija do $100 Hz$ te za otpor od $4.5 k\Omega$ i kapacitet kondenzatora od $7 \mu F$, izračunajte vrijednosti prijenosne funkcije.

In [ ]:
f = ...  # raspon frekvencija od 0 do 100 Hz

H = rc_lowpass(R, C, f)  # prijenosna funkcija

S obzirom da su vrijednosti prijenosne funkcije kompleksne veličine, razmilite što je potrebno napraviti s njima prije nego ih grafički prikažemo?

In [ ]:
Hm = ...  # konverzija u apsolutne vrijednosti

Vizualizirajte ovisnost prijenosne funkcije o frekvenciji koristeći `matplotlib` i funkciju `matplotlib.pyplot.plot`.

In [ ]:
plt.plot(...)  # prvi argument su vrijednosti na x-osi, drugi argument vrijednosti na y-osi
plt.xlabel('f')
plt.ylabel('H(f)')
plt.show()

Vizualizirajte sada rezultate koristeći već implementiranu funkciju `plot_frequency_response`.

Napomena: za provjeru načina korištenja prethodne funkcije koristite sljedeću naredbu:

```python
help(plot_frequency_response)
```

ili jednostavno

```python
plot_frequency_response?
```

In [ ]:
# provjerite način korištenja funkcije



In [ ]:
fig, ax = plot_frequency_response(...)  # grafički prikaz dobivenih rezultata

### Analiza spektra

Primjenom Fourierove transformacije, signal možemo promatrati u frekvencijskog domeni, tj., signal postaje funkcija frekvencije. Reprezentacija signala kroz frekvencijske komponente je posebice važna ukoliko na raspolaganju imamo mjerenja uz prisustvo velike količine šuma koji želimo filtrirati. U pravilu, frekvencijske komponente čistog signala su višestruko dominantnije u odnosu na komponente šuma.

#### Zadatak 3

Promotri funkciju generatora sinusnog signala uz prisustvo šuma `signal_samples` koja prima vremensku seriju trenutaka u kojima želimo generirati signal.

In [ ]:
def signal_samples(t):
    """Generator sinusnog signala uz prisustvo šuma.
    
    Args:
        t (number or numpy.ndarray) : vrijeme u kojem generiramo signal
    
    Returns:
        numpy.ndarray
    """
    signal = 2 * np.sin(2 * np.pi * t) + 3 * np.sin(22 * 2 * np.pi * t)
    noise = 2 * np.random.randn(*np.shape(t))
    return signal + noise

Imajući u vidu karakteristike zadanog signala, definiraj frekvenciju sempliranja, $f_s$, pazeći pritom na zadovoljavanje Nyquistovog teorema (frekvencija sempliranja mora biti barem 2 puta veća od najviše frekvencija signala). Postavi rezoluciju frekvencijskog spektra, $\Delta f$). na $0.01 Hz$, i prema tome definiraj ukupan broj semplova koji odgovara periodu signala ($T = N / f_s$).

In [ ]:
B = ...  # pojasna širina
fs = 2 * B  # frekvencija sempliranja
delta_f = ...  # rezolucija frekvencijskog spektra
N = int(...)  # broj semplova
T = ...  # period

Kreiraj vremensku domenu i definiraj vrijednosti signala za svaki od trenutaka.

In [ ]:
t = ...
f_t = signal_samples(t)

# vizual
fig, ax = plt.subplots(1, 2, figsize=(12, 3), sharey=True)
ax[0].plot(t, f_t)
ax[0].set_xlabel('$t$ [s]')
ax[0].set_ylabel('signal')
ax[1].plot(t, f_t)
ax[1].set_xlim(0, 5)
ax[1].set_xlabel('$t$ [s]');

Proračunaj spektralne komponente signala koristeći `np.fft.fft` funkciju.

In [ ]:
F = np.fft.fft(...)  # Fourierov transformat signala u vremenu
f = np.fft.fftfreq(N, 1.0/fs)  # odgovarajuće frekvencije
mask = np.where(f >= 0)  # s obzirom na simetričnost, promatramo samo pozitivne frekvencije

# vizual
fig, ax = plt.subplots(3, 1, figsize=(12, 6))
ax[0].plot(f[mask], np.log(abs(F[mask])), label='real')
ax[0].plot(B, 0, 'r*', markersize=10)
ax[0].set_ylabel('$\log(|F|)$', fontsize=14)
ax[1].plot(f[mask], abs(F[mask])/N, label='real')
ax[1].set_xlim(0, 2)
ax[1].set_ylabel('$|F|/N$', fontsize=14)
ax[2].plot(f[mask], abs(F[mask])/N, label='real')
ax[2].set_xlim(21, 23)
ax[2].set_xlabel('$f$ [Hz]')
ax[2].set_ylabel('$|F|/N$');

### Butterworthov filtar

Butterworthov filtar vrsta je filtra za obradu signala dizajnirana tako da je frekvencijski odziv što je moguće ravniji u propusnom opsegu. 

<center>
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/dc/LowPass3poleICauer.svg/1920px-LowPass3poleICauer.svg.png" alt="butterworth" width="500"/>
</center>

Funkcija prijenosnog odziva, $H$, niskopropusnog Butterworth filtra trećeg reda (Cauer, ljestvasta topologija), prikazanog na slici poviše, ima sljedeći oblik:

$$H(s) = \frac{V_{out}(s)}{V_{in}(s)} = \frac{R_4}{s^3(L_1 C_2 L_3) + s^2(L_1 C_2 R_4) + s(L_1 + L_3) + R_4}$$

Ukoliko zadržimo istu *cutoff* frekvenciju kao i u prvom primjeru, funkcija Butterworthovog filtra će, kao glavnu prednost, imati puno strmiju granicu propusnog i nepropusnog frekvencijskog područja (ovisno o redu filtra).

In [ ]:
f = np.linspace(0, 100, 10000)  # raspon frekvencija od 0 do 100 Hz
fc = cutoff_frequency(R, C)  # cutoff frekvencija
fs = 1000  # frekvencija uzorkovanja
fs_nyq = 0.5 * fs  # Nyquistova frekvencija
fc_norm = fc / fs_nyq  # normalizirana cutoff frekvencija
order = 3  # red filtra

b, a = ss.butter(order, fc_norm, btype='low')  # brojnik i nazivnik IIR filtra
f_norm, H = ss.freqz(b, a, worN=f.size)  # normalizirane frekvencije i prijenosna funkcija filtra
f_denorm = f_norm / np.pi * fs_nyq  # rekonstruirane frekvencije
Hm = abs(H)  # apsolutna vrijednost prijenosne funkcije filtra

In [ ]:
fig, ax = plot_frequency_response(f_denorm, Hm, fc, ylim_min=-80)  # grafički prikaz dobivenih rezultata

#### Zadatak 3

Implementiraj funkciju generatora sinusnog signala `signal_generator` koja prima vremensku seriju trenutaka u kojima želimo generirati signal, amplitudu signala, željenu frekvenciju signala, i razinu šuma koji se nadodaje signalu.

In [ ]:
def signal_generator(t, A, f, noise_factor):
    """Generator sinusnog signala.
    
    Args:
        t (number or numpy.ndarray) : vrijeme u kojem generiramo signal
        A (number) : amplituda signala
        f (number) : frekvencija signala
        noise_factor (number) : razina šuma 
    
    Returns:
        tuple
    """
    ######################################################
    ## TO-DO: implementiraj funkciju generatora signala ##
    # Nakon toga zakomentiraj sljedeću liniju.
    raise NotImplementedError('Implementiraj funkciju generatora signala.')
    ######################################################

    # definiraj sinusni signal amplitude `A` za svaki `t`
    signal = ...
    
    # definiraj bijeli šum skaliran za `noise_factor`
    noise = ...
    
    # konačni signal = suma signala i šuma
    noisy_signal = ...
    return signal, noisy_signal

Generiraj sinusni signal i signal uz prisustvo šuma frekvencije $1 Hz$ u trajanju od $5$ sekunda, koristeći frekvenciju uzorkovanja od $30 Hz$. Amplituda signala neka bude $2$ a amplituda šuma $1$.

In [ ]:
T = ...  # ukupno trajanje signala
fs = ...  # frekvencija uzorkovanja
A = ...  # amplituda signala
f = ...  # frekvencija signala
noise_factor = ...  # amplituda bijelog šuma
t = np.linspace(0, T, T * fs)  # trenuci u kojima generiramo signal

signal, noisy_signal = signal_generator(t, A, f, noise_factor)

Grafički prikaži ovisnost signala i signala uz prisustvo šuma u vremenu.

In [ ]:
plt.plot(..., label='originalni signal')
plt.plot(..., label='mjereni signal uz prisustvo šuma')
plt.xlabel('t')
plt.ylabel('signal')
plt.legend()
plt.grid()
plt.show()

In [ ]:
def butter_filter(noisy_signal, fc, fs, order=3):
    """Butterworthov nisko-propusni filtar.
    
    Args:
        noisy_signal (numpy.ndarray) : signal uz prisustvo šuma
        fc (number) : cutoff frekvencija
        fs (number) : frekvencija uzorkovanja signala
        order (int) : red filtra
    
    Returns:
        numpy.ndarray
    """
    fs_nyq = 0.5 * fs
    fc_norm = fc / fs_nyq
    b, a = ss.butter(order, fc_norm, btype='low')
    filtered_signal = ss.filtfilt(b, a, noisy_signal)
    return filtered_signal

Filtriraj signal uz prisustvo šuma koristeći funkciju `butter_filter`. Mijenjaj *cutoff* frekvenciju dok ne dobiješ zadovoljavajuću rekonstrukciju signala. Počni s vrijednošću dobivenu za RC nisko-propusni filtar iz prvog zadatka.

In [ ]:
fc = ...  # proizvoljna cutoff frekvencija

filtered_signal = butter_filter(noisy_signal, fc, fs)

Grafički prikaz ovisnost signala i signala uz prisustvo šuma u vremenu.

In [ ]:
plt.plot(t, signal, label='originalni signal')
plt.plot(t, noisy_signal, label='mjereni signal uz prisustvo šuma')
plt.plot(t, filtered_signal, label='filtrirani signal')
plt.xlabel('t')
plt.ylabel('signal')
plt.legend()
plt.grid()
plt.show()

Za koju *cutoff* frekvenciju dobivate optimalnu rekonstrukciju signala? Komentirajte rezultate.

Zaključak:
